In [0]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [0]:
from fastai.text import *

In [0]:
bs=64

In [4]:
path=untar_data(URLs.HUMAN_NUMBERS)
path.ls()

[PosixPath('/root/.fastai/data/human_numbers/valid.txt'),
 PosixPath('/root/.fastai/data/human_numbers/train.txt')]

In [0]:
def readnums(d): return [', '.join(o.strip() for o in open(path/d).readlines())]

In [6]:
train_txt=readnums('train.txt')
train_txt[0][:80]

'one, two, three, four, five, six, seven, eight, nine, ten, eleven, twelve, thirt'

In [7]:
valid_txt=readnums('valid.txt')
valid_txt[0][-80:]

' nine thousand nine hundred ninety eight, nine thousand nine hundred ninety nine'

In [0]:
train=TextList(train_txt,path=path)
valid=TextList(valid_txt,path=path)

In [0]:
src=ItemLists(path=path,train=train,valid=valid).label_for_lm()
data=src.databunch(bs=bs)

In [10]:
train[0].text[:80]  

'xxbos one , two , three , four , five , six , seven , eight , nine , ten , eleve'

In [11]:
len(data.valid_ds[0][0].data)

13017

In [12]:
data.bptt,len(data.valid_dl)

(70, 3)

In [13]:
13017/70/bs

2.905580357142857

In [0]:
it=iter(data.valid_dl)
x1,y1=next(it)
x2,y2=next(it)
x3,y3=next(it)

In [15]:
x1.numel()+x2.numel()+x3.numel()

13440

In [16]:
x1[:,0]

tensor([ 2,  9, 11, 12, 13, 11, 10,  9, 10, 14, 19, 25, 19, 15, 16, 11, 19,  9,
        10,  9, 19, 25, 19, 11, 19, 11, 10,  9, 19, 20, 11, 26, 20, 23, 20, 20,
        24, 20, 11, 14, 11, 11,  9, 14,  9, 20, 10, 20, 35, 17, 11, 10,  9, 17,
         9, 20, 10, 20, 11, 20, 11, 20, 20, 20], device='cuda:0')

In [0]:
v=data.valid_ds.vocab

In [18]:
v.textify(x1[0])

'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight'

In [20]:
data.show_batch(ds_type=DatasetType.Valid)

idx,text
0,"thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight thousand sixty , eight thousand sixty"
1,"eight , eight thousand eighty nine , eight thousand ninety , eight thousand ninety one , eight thousand ninety two , eight thousand ninety three , eight thousand ninety four , eight thousand ninety five , eight thousand ninety six , eight thousand ninety seven , eight thousand ninety eight , eight thousand ninety nine , eight thousand one hundred , eight thousand one hundred one , eight thousand one"
2,"thousand one hundred twenty four , eight thousand one hundred twenty five , eight thousand one hundred twenty six , eight thousand one hundred twenty seven , eight thousand one hundred twenty eight , eight thousand one hundred twenty nine , eight thousand one hundred thirty , eight thousand one hundred thirty one , eight thousand one hundred thirty two , eight thousand one hundred thirty three , eight thousand"
3,"three , eight thousand one hundred fifty four , eight thousand one hundred fifty five , eight thousand one hundred fifty six , eight thousand one hundred fifty seven , eight thousand one hundred fifty eight , eight thousand one hundred fifty nine , eight thousand one hundred sixty , eight thousand one hundred sixty one , eight thousand one hundred sixty two , eight thousand one hundred sixty three"
4,"thousand one hundred eighty three , eight thousand one hundred eighty four , eight thousand one hundred eighty five , eight thousand one hundred eighty six , eight thousand one hundred eighty seven , eight thousand one hundred eighty eight , eight thousand one hundred eighty nine , eight thousand one hundred ninety , eight thousand one hundred ninety one , eight thousand one hundred ninety two , eight thousand"


In [0]:
data=src.databunch(bs=bs,bptt=3)

In [22]:
x,y=data.one_batch()
x.shape,y.shape

(torch.Size([64, 3]), torch.Size([64, 3]))

In [65]:
nv=len(v.itos)
nv

40

In [0]:
nh=64

In [0]:
def loss4(input,target): return F.cross_entropy(input,target[:,-1])
def acc4(input,target): return accuracy(input,target[:,-1])

In [0]:
class Model0(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh) 
        self.h_h = nn.Linear(nh,nh)     
        self.h_o = nn.Linear(nh,nv)     
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = self.bn(F.relu(self.h_h(self.i_h(x[:,0]))))
        if x.shape[1]>1:
            h = h + self.i_h(x[:,1])
            h = self.bn(F.relu(self.h_h(h)))
        if x.shape[1]>2:
            h = h + self.i_h(x[:,2])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)
    

In [0]:
learn = Learner(data, Model0(), loss_func=loss4, metrics=acc4)

In [35]:
learn.fit_one_cycle(6,1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.661772,3.504384,0.167969,00:01
1,2.795001,2.735320,0.439798,00:01
2,2.191174,2.282206,0.453125,00:01
3,1.955117,2.143176,0.456342,00:01
4,1.872009,2.106331,0.462546,00:01
5,1.854391,2.101676,0.461397,00:01


##Putting it in a loop

In [0]:
class Model1(nn.Module):
  def __init__(self):
    super().__init__()
    self.i_h=nn.Embedding(nv,nh)
    self.h_h=nn.Linear(nh,nh)
    self.h_o=nn.Linear(nh,nv)
    self.bn=nn.BatchNorm1d(nh)
  def forward(self,x):
    h=torch.zeros(x.shape[0],nh).to(device=x.device)
    for i in range(x.shape[1]):
      h=h+self.i_h(x[:,i])
      h=self.bn(F.relu(self.h_h(h)))
    return self.h_o(h)


In [0]:
learn1=Learner(data, Model1(),loss_func=loss4, metrics=acc4)

In [43]:
learn1.fit_one_cycle(6,1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.535989,3.445286,0.147518,00:01
1,2.639313,2.564615,0.424862,00:01
2,2.057799,2.142698,0.464384,00:01
3,1.838730,2.033419,0.461397,00:01
4,1.764704,2.014163,0.454044,00:01
5,1.749445,2.012683,0.454733,00:01


##Multi Connected

In [0]:
data=src.databunch(bs=bs,bptt=20)

In [47]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 20]), torch.Size([64, 20]))

In [0]:
class Model2(nn.Module):
  def __init__(self):
    super().__init__()
    self.i_h=nn.Embedding(nv,nh)
    self.h_h=nn.Linear(nh,nh)
    self.h_o=nn.Linear(nh,nv)
    self.bn=nn.BatchNorm1d(nh)
  def forward(self,x):
    res=[]
    h=torch.zeros(x.shape[0],nh).to(device=x.device)
    for i in range(x.shape[1]):
      h=h+self.i_h(x[:,i])
      h=F.relu(self.h_h(h))
      res.append(self.h_o(self.bn(h)))
    return torch.stack(res,dim=1)


In [0]:
learn=Learner(data, Model2(),metrics=accuracy)

In [50]:
learn.fit_one_cycle(10,1e-4,pct_start=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,3.808314,3.732116,0.048793,00:00
1,3.616265,3.510117,0.104972,00:00
2,3.381063,3.307923,0.153906,00:00
3,3.147813,3.139293,0.244744,00:00
4,2.943619,3.007895,0.280256,00:00
5,2.781276,2.915414,0.299077,00:00
6,2.661966,2.857691,0.320028,00:00
7,2.581503,2.826980,0.335866,00:00
8,2.533000,2.815226,0.341406,00:00
9,2.507874,2.813478,0.342472,00:00


##Using nn.RNN

In [0]:
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.RNN(nh,nh, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(1, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [0]:
learn=Learner(data, Model4(),metrics=accuracy)

In [82]:
learn.fit_one_cycle(20,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.686913,1.568590,0.504616,00:00
1,0.664302,1.577441,0.499858,00:00
2,0.622471,1.457159,0.542685,00:00
3,0.570636,1.453682,0.567756,00:00
4,0.512549,1.392020,0.586364,00:00
5,0.445501,1.391629,0.588068,00:00
6,0.379254,1.271059,0.638494,00:00
7,0.324708,1.214593,0.661932,00:00
8,0.278757,1.226656,0.665270,00:00
9,0.242171,1.232507,0.667827,00:00


##GRU

In [0]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.GRU(nh,nh,2,batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(2, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [0]:
learn=Learner(data,Model5(),metrics=accuracy)

In [88]:
learn.fit_one_cycle(10,1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.684625,2.427687,0.452131,00:00
1,1.568711,1.394434,0.641832,00:00
2,0.752583,1.121034,0.781676,00:00
3,0.369034,1.190816,0.815057,00:00
4,0.190223,1.197254,0.821449,00:00
5,0.103317,1.190203,0.810298,00:00
6,0.060033,1.243554,0.816619,00:00
7,0.037170,1.254969,0.816974,00:00
8,0.024881,1.327061,0.820881,00:00
9,0.018244,1.293638,0.819176,00:00


In [98]:
path.ls()

[PosixPath('/root/.fastai/data/human_numbers/models'),
 PosixPath('/root/.fastai/data/human_numbers/valid.txt'),
 PosixPath('/root/.fastai/data/human_numbers/train.txt')]

In [106]:
learn.export()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Model5. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
